삼성 온라인쇼핑몰은 미국 아마존에 대항할 세계 최고의 온라인 쇼핑몰로 발돋움하기 위해 실력 있다는 Data Scientist를 총 집합시켜 데이터를 분석하기로 하였다.  삼성 온라인쇼핑몰로 초빙된 당신은 다음 데이터를 활용하여 분석을 하고자 한다.  

다음은 데이터 모델링을 위해 주어지는 파일 목록이다.<br>
<b>ds_purchase_log.csv</b> : 삼성 온라인쇼핑몰에서 8월 31일 하루 동안 발생한 구매 로그 데이터
<b>cust_no</b> : 회원번호(상품을 구매한 회원번호) <br>
<b>item_no</b> : 상품번호(회원이 구매한 상품번호) <br>
 
<b>ds_customer_mst.csv</b> : 회원 마스터로, 회원정보가 수정되면 기존 정보는 그대로 있고, 수정된 정 보가 추가된다. <br>
<b>cust_no</b> : 회원번호 <br>
<b>age_gender</b> : 회원 연령대, 성별(연령대: 20대, 30대, 40대, 50대 이상)(성별: M-남, F-여) <br>
<b>update_dt</b> : 수정일자 <br>
 
<b>ds_product_mst.csv</b> : 상품 마스터<br>
<b>item_no</b> : 상품번호 <br>
<b>item_ctg1</b> : 상품의 대카테고리 <br>
<b>item_ctg2</b> : 상품의 중카테고리 <br>
<b>item_amt</b> : 매출(구매금액) <br>
 
분석에 앞서, 다음의 전처리를 수행하여 base_table을 만든다.<br>
ds_purchase_log.csv 로그를 기준으로 ds_customer_mst.csv에서 회원 연령대, 성별(age_gender) 정 보를 가져오고, ds_product_mst.csv에서 카테고리(item_ctg1, item_ctg2)와 매출(item_amt) 정보를 가져온다.<br>
이 때, ds_customer_mst.csv에서 회원 연령대, 성별(age_gender)를 가져올 때, 고객별 가장 최신의 정보(고객별 update_dt가 가장 큰 정보)를 가져온다. 단, 동일 회원번호에 대해 update_dt가 동일 한데, age_gender 컬럼의 값이 다른 경우, max(age_gender)값으로 가져오시오. 

In [4]:
import pandas as pd
import numpy as np

## 데이터 준비

In [72]:
log_df = pd.read_csv('ds_purchase_log.csv')
customer_df = pd.read_csv('ds_customer_mst.csv')
product_df = pd.read_csv('ds_product_mst.csv')

In [80]:
log_df.describe()

,cust_no,item_no
count,27910.000000,27910.000000
mean,9612.280652,1306.250305
std,5499.154031,778.255095
min,1.000000,1.000000
25%,4925.250000,607.000000
50%,9529.500000,1302.000000
75%,14368.750000,1989.000000
max,19111.000000,2631.000000


In [34]:
customer_df.head()

,cust_no,age_gender,update_dt
0,695,20대 F,20180825
1,12875,20대 M,20180825
2,12971,20대 F,20180825
3,14991,50대이상 M,20180825
4,14991,50대이상 M,20180825


In [35]:
product_df.head()

,item_no,item_ctg1,item_ctg2,item_amt
0,1,식품,즉석식품,34500.0
1,2,생활용품,화장지,21100.0
2,3,의류,잠옷,72100.0
3,4,의류,임부복,61000.0
4,5,화장품,클렌징,14500.0


In [73]:
n_customer_df = customer_df.join(customer_df['age_gender'].str.split(' ', 1, expand=True).rename(columns={0:'age', 1:'gender'}))

In [74]:
n_customer_df = n_customer_df[['cust_no', 'age', 'gender', 'update_dt', 'age_gender']]
n_customer_df.head()

,cust_no,age,gender,update_dt,age_gender
0,695,20대,F,20180825,20대 F
1,12875,20대,M,20180825,20대 M
2,12971,20대,F,20180825,20대 F
3,14991,50대이상,M,20180825,50대이상 M
4,14991,50대이상,M,20180825,50대이상 M


In [75]:
n_customer_df['age'].replace(regex=True,
    to_replace=['대', '이상'],
    value='',
    inplace=True
)
n_customer_df.head()

,cust_no,age,gender,update_dt,age_gender
0,695,20,F,20180825,20대 F
1,12875,20,M,20180825,20대 M
2,12971,20,F,20180825,20대 F
3,14991,50,M,20180825,50대이상 M
4,14991,50,M,20180825,50대이상 M


In [140]:
n_customer_df = n_customer_df.sort_values(by=['cust_no', 'age_gender', 'update_dt'])
basic_cm_df = n_customer_df.drop_duplicates(subset='cust_no', keep='last')
basic_cm_df.head()

,cust_no,age,gender,update_dt,age_gender
3191,1,40,M,20180829,40대 M
1684,2,40,M,20180829,40대 M
4710,3,40,F,20180831,40대 F
5923,4,20,F,20180831,20대 F
5114,5,40,F,20180831,40대 F


In [141]:
del basic_cm_df['age_gender']

In [142]:
a = pd.merge(log_df, basic_cm_df, on='cust_no')
basic_table = pd.merge(a, product_df, on='item_no')
basic_table.head()

,cust_no,item_no,age,gender,update_dt,item_ctg1,item_ctg2,item_amt
0,1,1322,40,M,20180829,의류,남성의류,94900.0
1,804,1322,40,M,20180831,의류,남성의류,94900.0
2,1062,1322,40,M,20180831,의류,남성의류,94900.0
3,1134,1322,30,F,20180831,의류,남성의류,94900.0
4,1457,1322,40,M,20180831,의류,남성의류,94900.0


In [143]:
basic_table.describe()

,cust_no,item_no,update_dt,item_amt
count,27910.000000,27910.000000,2.791000e+04,27910.000000
mean,9612.280652,1306.250305,2.018083e+07,60249.684701
std,5499.154031,778.255095,7.791361e-01,65864.865884
min,1.000000,1.000000,2.018083e+07,4900.000000
25%,4925.250000,607.000000,2.018083e+07,33600.000000
50%,9529.500000,1302.000000,2.018083e+07,46100.000000
75%,14368.750000,1989.000000,2.018083e+07,66300.000000
max,19111.000000,2631.000000,2.018083e+07,809400.000000


In [144]:
basic_table = basic_table.sort_values(by='cust_no')

## 1번 문제
(base_table을 활용하여 분석을 수행하시오.) 성별에 따라 어떤 대카테고리에 더 많은 매출이 발생하는지 비교하고자 한다.<br>
매출의 비교는 대카테고리별 인당 매출금액 기준으로 하며, 인당 매출금액은 매출 총 금액을 매 출을 발생시킨 회원수를 나눈 값으로 한다. 남녀 간 비교를 위해 회원 연령대, 성별(age_gender) 을 활용하여 성별 컬럼을 만들고, 성별 대카테고리별 인당 구매금액을 산출하여, 남성 1인당 매출 액 2위인 카테고리(A)와, 여성 1인당 매출액 2위인 카테고리(B)를 차례로 쓰시오.<br>
(답안예시) 가전,생활용품 

In [145]:
men_df = basic_table[basic_table['gender'] == 'M']
women_df = basic_table[basic_table['gender'] == 'F']
men_df.head()

,cust_no,item_no,age,gender,update_dt,item_ctg1,item_ctg2,item_amt
0,1,1322,40,M,20180829,의류,남성의류,94900.0
89,2,2275,40,M,20180829,의류,유아의류,36700.0
40,2,2273,40,M,20180829,의류,남성의류,99900.0
392,6,461,50,M,20180831,의류,기타의류,51200.0
427,7,833,20,M,20180831,식품,과자류,10100.0


In [146]:
men_ctg1_sum = men_df['item_amt'].groupby(men_df['item_ctg1']).sum()
men_ctg1_sum

item_ctg1
가전      130788200.0
생활용품     46897300.0
식품       67395700.0
의류      604807600.0
화장품      23177800.0
Name: item_amt, dtype: float64

In [147]:
men_ctg1_num = men_df.drop_duplicates(subset='cust_no')['cust_no'].groupby(men_df['item_ctg1']).count()
men_ctg1_num

item_ctg1
가전       209
생활용품    1269
식품      1377
의류      6836
화장품      216
Name: cust_no, dtype: int64

In [148]:
result = men_ctg1_sum / men_ctg1_num
men_rank = result.sort_values(ascending=False)
men_rank

item_ctg1
가전      625780.861244
화장품     107304.629630
의류       88473.902867
식품       48943.863471
생활용품     36956.107171
dtype: float64

In [149]:
women_ctg1_sum = women_df['item_amt'].groupby(women_df['item_ctg1']).sum()
women_ctg1_sum

item_ctg1
가전      119879200.0
생활용품     39827200.0
식품       67924300.0
의류      560024200.0
화장품      20847200.0
Name: item_amt, dtype: float64

In [150]:
women_ctg1_num = women_df.drop_duplicates(subset='cust_no')['cust_no'].groupby(women_df['item_ctg1']).count()
women_ctg1_num

item_ctg1
가전       189
생활용품    1002
식품      1338
의류      6431
화장품      194
Name: cust_no, dtype: int64

In [151]:
result = women_ctg1_sum / women_ctg1_num
women_rank = result.sort_values(ascending=False)
women_rank

item_ctg1
가전      634281.481481
화장품     107459.793814
의류       87081.977919
식품       50765.545590
생활용품     39747.704591
dtype: float64

## 2번 문제

(base_table 또는 ds_purchase_log.csv 을 활용하여 분석을 수행하시오.) 상위 20퍼센트의 회원 이 일으킨 매출은 전체 매출의 몇 퍼센트를 차지하며, 상위 20퍼센트의 회원이 일으킨 매출은 하 위 20퍼센트의 회원이 일으킨 매출의 몇 배인지 분석하고자 한다.<br>
상위 20퍼센트와 하위 20퍼센트를 산정하는 방식은 다음과 같다. 
 
(a) 전체 회원에 대해 회원별 구매금액에 대해 내림차순으로 순위를 매긴다. 순위는 동차발생시 동차로 등수를 주고 그 다음 순위는 동차 사람수만큼 더한 이후 그 다음 순위를 준다 (아래 예시 참조)

<center><순위 예시></center>

|  <center>금액</center> | <center>순위</center> |
|:--------:|:--------:|
|<center>5300</center>|<center>1</center> |
|<center>5300</center>|<center>1</center>|
|<center>5300</center>|<center>1</center>|
|<center>5200</center>|<center>4</center>|
|<center>5200</center>|<center>4</center>|


(b) 상위 20퍼센트는 Rank <= N*0.2 인 회원을 의미하며, 하위 20퍼센트는 Rank >= N*0.8 인 회 원을 의미한다. (여기서 N은 base_table 또는 ds_purchase_log.csv에 구매이력이 존재하는 회원수, Rank는 위 (a)에서 만든 순위를 의미한다) 
 
상위 20퍼센트의 회원이 일으킨 매출은 전체 매출의 몇 퍼센트(A)이며, 상위 20퍼센트의 회원이 일으킨 매출은 하위 20퍼센트의 회원이 일으킨 매출의 몇 배(B) 인지 차례대로 쓰시오.
 
- Hint : Brightics는 hive 구문 rank() over(order by) 활용
- A, B는 소수점 아래는 버리고, 정수형으로 기술하시오. 
 
(답안예시) 15,2

In [166]:
basic_table.head(10)

,cust_no,item_no,age,gender,update_dt,item_ctg1,item_ctg2,item_amt
0,1,1322,40,M,20180829,의류,남성의류,94900.0
89,2,2275,40,M,20180829,의류,유아의류,36700.0
40,2,2273,40,M,20180829,의류,남성의류,99900.0
102,3,1309,40,F,20180831,식품,간식,26000.0
120,3,605,40,F,20180831,의류,남성의류,98600.0
169,4,1411,20,F,20180831,의류,유아의류,42000.0
177,5,898,40,F,20180831,생활용품,화장지,20600.0
324,5,1272,40,F,20180831,의류,남성의류,98700.0
359,5,455,40,F,20180831,의류,남성의류,94000.0
392,6,461,50,M,20180831,의류,기타의류,51200.0


In [209]:
purchase = basic_table.sort_values(by='cust_no')
sorted_purchase = purchase['item_amt'].groupby(purchase['cust_no']).sum().sort_values(ascending=False)

In [198]:
def rank(dic):
    rank_list, b_v, cnt = [], 0, 0
    for k, v in dic.items():
        if not rank_list:
            rank_list.append(1)
            cnt += 1
        else:
            if b_v == v:
                rank = rank_list[-1]
                cnt += 1
            else:            
                rank = rank_list[-1] + cnt
                cnt = 1
            rank_list.append(rank)
        b_v = v
    return rank_list

In [200]:
rank({1:10, 2:10, 3:10, 4:11, 5:11, 6:12, 7:13, 8:13, 9:14, 10:15})

[1, 1, 1, 4, 4, 6, 7, 7, 9, 10]

In [214]:
N = len(sorted_purchase)

In [212]:
sp = pd.DataFrame(sorted_purchase)
sp['rank'] = rank(sorted_purchase)

In [213]:
sp.head()

,item_amt,rank
cust_no,,
6932,2394000.0,1
4401,1978900.0,2
2968,1930700.0,3
2223,1795200.0,4
12049,1716000.0,5


In [218]:
# 상위 20%
high_20 = sp[sp['rank'] <= N * 0.2]

# 하위 20% 
low_20 = sp[sp['rank'] >= N * 0.8]

In [221]:
high_20.head()

,item_amt,rank
cust_no,,
6932,2394000.0,1
4401,1978900.0,2
2968,1930700.0,3
2223,1795200.0,4
12049,1716000.0,5


In [227]:
high_20_sale = high_20['item_amt'].sum()
low_20_sale = low_20['item_amt'].sum()
total_sale = purchase['item_amt'].sum()
print(high_20_sale, low_20_sale, total_sale)

826011100.0 90639900.0 1681568700.0


In [228]:
a = high_20_sale / total_sale * 100
a

49.121460217474315

In [230]:
b = high_20_sale / low_20_sale
b

9.113106920903487

## 3번 문제

 (base_table을 활용하여 분석을 수행하시오.) 회원별 구매금액이 연령대별로 차이가 있는지 비 교하고자 회원 연령대, 성별(age_gender)을 활용하여 연령대(age_cls) 컬럼을 만들고 다음 분석을 수행한다.<br>
 각 회원별 구매한 총 구매금액(y)을 산출하고, 회원의 연령대 그룹을 매핑한다. 연령대 그룹(x)은 age_cls가 '30대'이면 그룹이 ‘G1’, age_cls가 '40대'이면 그룹이 ‘G2’, age_cls가 그 외 연령대이면 그룹이 ‘G3’이다.<br>
 연령대 그룹(x)에 따라 회원별 구매한 총 구매금액(y)이 차이가 있는지 ANOVA를 수행하고, ANOVA의 F-통계량 추정치(A)를 구하시오. 또한, 사후검정으로 Duncan Test(alpha=0.05)를 수행하 여 평균구매금액 순으로 오름차순하고, 그 결과를 등호, 부등호로 표기하시오.
 
- A는 소수점 아래는 버리고, 정수형으로 기술하시오.
- B는 회원별 구매한 총 구매금액(y)의 평균이 그룹 G1, G2, G3 순으로 크고, Duncan Test 결과가 1, 2가 같다면 G1=G2>G3으로 표기한다.
 
(답안예시) 3,G1=G2>G3

In [378]:
basic_table.head()

,cust_no,item_no,age,gender,update_dt,item_ctg1,item_ctg2,item_amt
0,1,1322,40,M,20180829,의류,남성의류,94900.0
89,2,2275,40,M,20180829,의류,유아의류,36700.0
40,2,2273,40,M,20180829,의류,남성의류,99900.0
102,3,1309,40,F,20180831,식품,간식,26000.0
120,3,605,40,F,20180831,의류,남성의류,98600.0


In [380]:
G1 = basic_table[basic_table['age'] == 30]
G2 = basic_table[basic_table['age'] == 40]

In [393]:
a = basic_table[basic_table['age'].astype(int) < 30]
b = basic_table[basic_table['age'].astype(int) > 40]
G3 = pd.concat([a, b])
G3.head()

,cust_no,item_no,age,gender,update_dt,item_ctg1,item_ctg2,item_amt
169,4,1411,20,F,20180831,의류,유아의류,42000.0
427,7,833,20,M,20180831,식품,과자류,10100.0
452,7,2264,20,M,20180831,의류,여성의류,61400.0
407,7,2514,20,M,20180831,의류,아동/주니어의류,42400.0
441,7,2471,20,M,20180831,의류,기타의류,47500.0


## 4번 문제
(base_table을 활용하여 분석을 수행하시오.) 함께 구매되는 상품이 어떤지 분석하여 상품 상세 페이지에 ‘함께 구매하는 상품’으로 추천하기 위해, 구매 로그의 중카테고리 기준으로 연관성 분 석(Association Rule)을 수행하고자 한다. 하나의 중카테고리만 구매한 이력이 있는 회원은 제외하 고, 둘 이상의 중카테고리를 구매한 회원의 구매 이력을 기반으로 연관성 분석을 수행하고, 중카 테고리가 ‘농산물’인 상품의 상세 페이지에 함께 노출되면 좋을 Lift 기준의 가장 적합한 중카테고 리를 구하시오. (단, Min Support는 0.001 이상, Min Confidence는 0.001 이상 이어야 한다.)
 
(답안예시) 스포츠의류

In [1]:
import apyori

In [319]:
basic_table.head()

,cust_no,item_no,age,gender,update_dt,item_ctg1,item_ctg2,item_amt
0,1,1322,40,M,20180829,의류,남성의류,94900.0
89,2,2275,40,M,20180829,의류,유아의류,36700.0
40,2,2273,40,M,20180829,의류,남성의류,99900.0
102,3,1309,40,F,20180831,식품,간식,26000.0
120,3,605,40,F,20180831,의류,남성의류,98600.0


In [353]:
hist = basic_table['item_ctg2'].groupby(basic_table['cust_no']).count()
b = basic_table[['cust_no', 'item_ctg2']].sort_values(by='cust_no')
b.head()

,cust_no,item_ctg2
0,1,남성의류
89,2,유아의류
40,2,남성의류
102,3,간식
120,3,남성의류


In [354]:
t = pd.DataFrame(hist.index, columns=['cust_no', 'freq'])
t['freq'] = hist.values

In [355]:
t.head()

,cust_no,freq
0,1,1
1,2,2
2,3,2
3,4,1
4,5,3


In [358]:
result = pd.merge(b, t, on='cust_no')
last = result[result['freq'] > 1]

In [367]:
unique = last.drop_duplicates('cust_no')
N = len(unique)

In [372]:
collection = []
for x in unique['cust_no']:
    collection.append(last[last['cust_no'] == x]['item_ctg2'].tolist())

In [373]:
rules = apyori.apriori(collection, min_support = 0.001, min_confidence = 0.001, min_lift = 3, min_length = 2)

In [ ]:
results = list(rules)

In [375]:
results

[RelationRecord(items=frozenset({'농산물', '가공식품'}), support=0.004843396835647401, ordered_statistics=[OrderedStatistic(items_base=frozenset({'가공식품'}), items_add=frozenset({'농산물'}), confidence=0.2777777777777778, lift=7.448292448292448), OrderedStatistic(items_base=frozenset({'농산물'}), items_add=frozenset({'가공식품'}), confidence=0.12987012987012986, lift=7.448292448292448)]),
 RelationRecord(items=frozenset({'수산축산물', '가공식품'}), support=0.0017759121730707135, ordered_statistics=[OrderedStatistic(items_base=frozenset({'가공식품'}), items_add=frozenset({'수산축산물'}), confidence=0.10185185185185186, lift=3.5642393806235617), OrderedStatistic(items_base=frozenset({'수산축산물'}), items_add=frozenset({'가공식품'}), confidence=0.06214689265536723, lift=3.5642393806235617)]),
 RelationRecord(items=frozenset({'수산축산물', '농산물'}), support=0.0033903777849531806, ordered_statistics=[OrderedStatistic(items_base=frozenset({'농산물'}), items_add=frozenset({'수산축산물'}), confidence=0.09090909090909091, lift=3.1813045711350796), Orde

In [376]:
def summary_apriori(results):

    summary_df = pd.DataFrame(columns=('Items','Antecedent','Consequent','Support','Confidence','Lift'))

    Support =[]
    Confidence = []
    Lift = []
    Items = []
    Antecedent = []
    Consequent=[]

    for RelationRecord in results: 
        for ordered_stat in RelationRecord.ordered_statistics:
            Support.append(RelationRecord.support)
            Items.append(RelationRecord.items)
            Antecedent.append(ordered_stat.items_base)
            Consequent.append(ordered_stat.items_add)
            Confidence.append(ordered_stat.confidence)
            Lift.append(ordered_stat.lift)

    summary_df['Items'] = Items                                   
    summary_df['Antecedent'] = Antecedent
    summary_df['Consequent'] = Consequent
    summary_df['Support'] = Support
    summary_df['Confidence'] = Confidence
    summary_df['Lift']= Lift
    
    return(summary_df)

In [377]:
summary_df=summary_apriori(results).sort_values('Lift',ascending=False)
summary_df.head(n=2)

,Items,Antecedent,Consequent,Support,Confidence,Lift
0,"(농산물, 가공식품)",(가공식품),(농산물),0.004843,0.277778,7.448292
1,"(농산물, 가공식품)",(농산물),(가공식품),0.004843,0.129870,7.448292
